# Imports

In [ ]:
import os
import argparse
import matplotlib
import numpy as np
from os import listdir
from time import sleep
import tensorflow as tf
from nltk import word_tokenize
from nltk import RegexpTokenizer
from os.path import isfile, join
import matplotlib.pyplot as plt
from tensorflow import keras as K
from progressbar import ProgressBar

tokenizer = RegexpTokenizer(r'\w+')
matplotlib.use('Agg')

### import Train data

In [ ]:
dictionary = np.load("./dataset/imdb/nltk_dictionary.npy").item()

In [ ]:
pos_files = [f for f in listdir("./dataset/imdb/train/pos") if ( isfile(join("./dataset/imdb/train/pos", f)) and (f[0] != '.') ) ]
neg_files = [f for f in listdir("./dataset/imdb/train/neg") if ( isfile(join("./dataset/imdb/train/neg", f)) and (f[0] != '.') ) ]
unl_files = [f for f in listdir("./dataset/imdb/train/unsup") if ( isfile(join("./dataset/imdb/train/unsup", f)) and (f[0] != '.') ) ]

#### Fine tuning

In [ ]:

print( "Labeled..." )
xtrain = list()
ytrain = list()
for p in pos_files:
    pos = open("./dataset/imdb/train/pos/"+p, 'r')
    for line in pos:
        if line:
            line = line.replace('<br /><br />', ' ')
            tokens = tokenizer.tokenize(line.lower())
            tokens = [ (dictionary[t] if (t in dictionary) else dictionary['<unk>']) for t in tokens ] 
            
            xtrain.append(tokens)
            ytrain.append(1)     
        else:
            print("empty line: {}.".format(line))
    pos.close()
    
for n in neg_files:
    neg = open("./dataset/imdb/train/neg/"+n, 'r')
    for line in neg:
        if line:
            line = line.replace('<br /><br />', ' ')
            tokens = tokenizer.tokenize(line.lower())
            tokens = [ (dictionary[t] if (t in dictionary) else dictionary['<unk>']) for t in tokens ] 
            
            xtrain.append(tokens)
            ytrain.append(0)       
        else:
            print("empty line: {}.".format(line))
    neg.close()

print( "Unlabeled..." )
unlab = list()
for u in unl_files:
    unl = open("./dataset/imdb/train/unsup/"+u, 'r')
    for line in unl:
        if line:
            line = line.replace('<br /><br />', ' ')
            tokens = tokenizer.tokenize(line.lower())
            tokens = [ (dictionary[t] if (t in dictionary) else dictionary['<unk>']) for t in tokens ] 
            
            unlab.append(tokens)            
        else:
            print("empty line: {}.".format(line))
    unl.close()

In [ ]:
xtrain = np.asarray(xtrain)
ytrain = np.asarray(ytrain)
unlab = np.asarray(unlab)

In [ ]:
np.save("./dataset/imdb/nltk_xtrain.npy", xtrain)
np.save("./dataset/imdb/nltk_ytrain.npy", ytrain)
np.save("./dataset/imdb/nltk_ultrain.npy", unlab)

### Import test data

In [ ]:
pos_files = [f for f in listdir("./dataset/imdb/test/pos") if ( isfile(join("./dataset/imdb/test/pos", f)) and (f[0] != '.') ) ]
neg_files = [f for f in listdir("./dataset/imdb/test/neg") if ( isfile(join("./dataset/imdb/test/neg", f)) and (f[0] != '.') ) ]

In [ ]:
xtest = list()
ytest = list()
for p in pos_files:
    pos = open("./dataset/imdb/test/pos/"+p, 'r')
    for line in pos:
        if line:
            line = line.replace('<br /><br />', ' ')
            tokens = tokenizer.tokenize(line.lower())
            tokens = [ (dictionary[t] if (t in dictionary) else dictionary['<unk>']) for t in tokens ] 
            
            xtest.append(tokens)
            ytest.append(1)
                
        else:
            print("empty line: {}.".format(line))
    pos.close()
    
for n in neg_files:
    neg = open("./dataset/imdb/test/neg/"+n, 'r')
    for line in neg:
        if line:
            line = line.replace('<br /><br />', ' ')
            tokens = tokenizer.tokenize(line.lower())
            tokens = [ (dictionary[t] if (t in dictionary) else dictionary['<unk>']) for t in tokens ] 
            
            xtest.append(tokens)
            ytest.append(0)
                
        else:
            print("empty line: {}.".format(line))
    neg.close()

In [ ]:
xtest = np.asarray(xtest)
ytest = np.asarray(ytest)

In [ ]:
np.save("./dataset/imdb/nltk_xtest.npy", xtest)
np.save("./dataset/imdb/nltk_ytest.npy", ytest)

In [ ]:
max( [len(s) for s in x] )

### Pre-trained glove embedding

In [ ]:
old_path = "./dataset/imdb/train/"
merged = open("./merged.txt", "w")

In [ ]:
current_directory = old_path + 'pos'
files = [f for f in listdir(current_directory) if ( isfile(join(current_directory, f)) and (f[0] != '.') ) ]
for f in files:
    t = open(current_directory+'/'+f, 'r')
    for line in t:
        if line:
            line = line.replace('<br /><br />', ' ')
            merged.write(line+'\n')
    t.close()

current_directory = old_path + 'neg'
files = [f for f in listdir(current_directory) if ( isfile(join(current_directory, f)) and (f[0] != '.') ) ]
for f in files:
    t = open(current_directory+'/'+f, 'r')
    for line in t:
        if line:
            line = line.replace('<br /><br />', ' ')
            merged.write(line+'\n')
    t.close()

current_directory = old_path + 'unsup'
files = [f for f in listdir(current_directory) if ( isfile(join(current_directory, f)) and (f[0] != '.') ) ]
for f in files:
    t = open(current_directory+'/'+f, 'r')
    for line in t:
        if line:
            line = line.replace('<br /><br />', ' ')
            merged.write(line+'\n')
    t.close()

merged.close()

In [ ]:
corpus = open("./merged.txt", "r")
new_corpus = open("./GloVe-1.2/new_merged", "w")

In [ ]:
lines = corpus.readlines()
text = ''

for line in lines:
    text = text + line
corpus.close()

words = tokenizer.tokenize(text.lower())

for w in words:
    new_corpus.seek(0, 2)
    new_corpus.write(w + ' ')
new_corpus.close()

In [ ]:
word_embedding = dict()
dictionary = dict()
vectors = open("./GloVe-1.2/vectors.txt", "r")
for i, line in enumerate(vectors):
    sline = line.split(' ')
    try:
        word = sline[0]
        word_embedding[word] = np.asarray(sline[1:], dtype='float32')
        dictionary[word] = i+1
    except:
        print("error at index {}".format(i))

In [ ]:
embedding = np.zeros( shape=(len(dictionary.keys())+1, len(list(word_embedding.values())[0])))
for word in dictionary.keys():
    idx = dictionary[word]
    embedding[idx] = word_embedding[word]

In [ ]:
np.save("./dataset/imdb/nltk_embedding_matrix.npy", embedding)
np.save("./dataset/imdb/nltk_dictionary.npy", dictionary)
np.save("./dataset/imdb/nltk_word_embedding.npy", word_embedding)

### Paper implementation

In [ ]:
class Network:
    def __init__(self, session, dict_weight, dropout=0.2, lstm_units=1024, dense_units=30):
        self.sess = session
        K.backend.set_session(self.sess)
        #defining layers
        dict_shape = dict_weight.shape
        self.emb = K.layers.Embedding(dict_shape[0], dict_shape[1], weights=[dict_weight], trainable=False, name='embedding')
        self.drop = K.layers.Dropout(rate=dropout, seed=91, name='dropout')
        self.lstm = K.layers.LSTM(lstm_units, stateful=False, return_sequences=False, name='lstm')
        self.dense = K.layers.Dense(dense_units, activation='relu', name='dense')
        self.p = K.layers.Dense(1, activation='sigmoid', name='p')
        #defining optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)
        # self.optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

    def __call__(self, batch, perturbation=None):
        embedding = self.emb(batch) 
        drop = self.drop(embedding)
        if (perturbation is not None):
            drop += perturbation
        lstm = self.lstm(drop)
        dense = self.dense(lstm)
        return self.p(dense), embedding
    
    def get_minibatch(self, x, y, ul, batch_shape=(64, 400)):
        x = K.preprocessing.sequence.pad_sequences(x, maxlen=batch_shape[1])
        permutations = np.random.permutation( len(y) )
        ul_permutations = None
        len_ratio = None
        if (ul is not None):
            ul = K.preprocessing.sequence.pad_sequences(ul, maxlen=batch_shape[1])
            ul_permutations = np.random.permutation( len(ul) )
            len_ratio = len(ul)/len(y)
        for s in range(0, len(y), batch_shape[0]):
            perm = permutations[s:s+batch_shape[0]]
            minibatch = {'x': x[perm], 'y': y[perm]}
            if (ul is not None):
                ul_perm = ul_permutations[int(np.floor(len_ratio*s)):int(np.floor(len_ratio*(s+batch_shape[0])))]
                minibatch.update( {'ul': np.concatenate((ul[ul_perm], x[perm]), axis=0)} )
            yield minibatch
            
    def get_loss(self, batch, labels):
        pred, emb = self(batch)
        loss = K.losses.binary_crossentropy(labels, pred)
        return tf.reduce_mean( loss ), emb
    
    def get_adv_loss(self, batch, labels, loss, emb, p_mult):
        gradient = tf.gradients(loss, emb, aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
        p_adv = p_mult * tf.nn.l2_normalize(tf.stop_gradient(gradient), dim=1)
        adv_loss = K.losses.binary_crossentropy(labels, self(batch, p_adv)[0])
        return tf.reduce_mean( adv_loss )
    
    def get_v_adv_loss(self, ul_batch, p_mult, power_iterations=1):
        bernoulli = tf.distributions.Bernoulli
        prob, emb = self(ul_batch)
        prob = tf.clip_by_value(prob, 1e-7, 1.-1e-7)
        prob_dist = bernoulli(probs=prob)
        #generate virtual adversarial perturbation
        d = tf.random_uniform(shape=tf.shape(emb), dtype=tf.float32)
        for _ in range( power_iterations ):
            d = (0.02) * tf.nn.l2_normalize(d, dim=1)
            p_prob = tf.clip_by_value(self(ul_batch, d)[0], 1e-7, 1.-1e-7)
            kl = tf.distributions.kl_divergence(prob_dist, bernoulli(probs=p_prob), allow_nan_stats=False)
            gradient = tf.gradients(kl, [d], aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
            d = tf.stop_gradient(gradient)
        d = p_mult * tf.nn.l2_normalize(d, dim=1)
        tf.stop_gradient(prob)
        #virtual adversarial loss
        p_prob = tf.clip_by_value(self(ul_batch, d)[0], 1e-7, 1.-1e-7)
        v_adv_loss = tf.distributions.kl_divergence(prob_dist, bernoulli(probs=p_prob), allow_nan_stats=False)
        return tf.reduce_mean( v_adv_loss )

    def validation(self, x, y, batch_shape=(64, 400)):
        print( 'Validation...' )
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='validation_labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='validation_batch')

        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        
        accuracies = list()
        minibatch = self.get_minibatch(x, y, ul=None, batch_shape=batch_shape)
        for val_batch in minibatch:
            fd = {batch: val_batch['x'], labels: val_batch['y'], K.backend.learning_phase(): 0} #test mode
            accuracies.append( self.sess.run(accuracy, feed_dict=fd) )
        
        print( "Average accuracy on validation is {:.3f}".format(np.asarray(accuracies).mean()) )
    
    def train(self, dataset, batch_shape=(64, 400), epochs=10, loss_type='none', p_mult=0.02, init=None, save=None):
        print( 'Training...' )
        xtrain = np.load( "{}nltk_xtrain.npy".format(dataset) )
        ytrain = np.load( "{}nltk_ytrain.npy".format(dataset) )
        ultrain = np.load( "{}nltk_ultrain.npy".format(dataset) ) if (loss_type == 'v_adv') else None
        
        # defining validation set
        xval = list()
        yval = list()
        for _ in range( int(len(ytrain)*0.025) ):
            xval.append( xtrain[0] ); xval.append( xtrain[-1] )
            yval.append( ytrain[0] ); yval.append( ytrain[-1] )
            xtrain = np.delete(xtrain, 0); xtrain = np.delete(xtrain, -1)
            ytrain = np.delete(ytrain, 0); ytrain = np.delete(ytrain, -1)
        xval = np.asarray(xval)
        yval = np.asarray(yval)
        print( '{} elements in validation set'.format(len(yval)) )
        # ---
        yval = np.reshape(yval, newshape=(yval.shape[0], 1))
        ytrain = np.reshape(ytrain, newshape=(ytrain.shape[0], 1))
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='train_labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='train_batch')
        ul_batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='ul_batch')
        
        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        loss, emb = self.get_loss(batch, labels)
        if (loss_type == 'adv'):
            loss += self.get_adv_loss(batch, labels, loss, emb, p_mult)
        elif (loss_type == 'v_adv'):
            loss += self.get_v_adv_loss(ul_batch, p_mult)

        opt = self.optimizer.minimize( loss )
        #initializing parameters
        if (init is None):
            self.sess.run( [var.initializer for var in tf.global_variables() if not('embedding' in var.name)] )
            print( 'Random initialization' )
        else:
            saver = tf.train.Saver()
            saver.restore(self.sess, init)
            print( 'Restored value' )
        
        _losses = list()
        _accuracies = list()
        list_ratio = (len(ultrain)/len(ytrain)) if (ultrain is not None) else None
        for epoch in range(epochs):
            losses = list()
            accuracies = list()
            validation = list()
            
            bar = ProgressBar(max_value=np.floor(len(ytrain)/batch_shape[0]).astype('i'))
            minibatch = enumerate(self.get_minibatch(xtrain, ytrain, ultrain, batch_shape=batch_shape))
            for i, train_batch in minibatch:
                fd = {batch: train_batch['x'], labels: train_batch['y'], K.backend.learning_phase(): 1} #training mode
                if (loss_type == 'v_adv'):
                    fd.update( {ul_batch: train_batch['ul']} )
                
                _, acc_val, loss_val = self.sess.run([opt, accuracy, loss], feed_dict=fd)
                
                accuracies.append( acc_val )
                losses.append( loss_val )
                bar.update(i)
            
            #saving accuracies and losses
            _accuracies.append( accuracies )
            _losses.append(losses)
            
            log_msg = "\nEpoch {} of {} -- average accuracy is {:.3f} (train) -- average loss is {:.3f}"
            print( log_msg.format(epoch+1, epochs, np.asarray(accuracies).mean(), np.asarray(losses).mean()) )
            
            # validation log
            self.validation(xval, yval, batch_shape=batch_shape)
            
            #saving model
            if (save is not None) and (epoch == (epochs-1)):
                saver = tf.train.Saver()
                saver.save(self.sess, save)
                print( 'model saved' )
        
        plt.plot([np.asarray(l).mean() for l in _losses], color='red', linestyle='solid', marker='o', linewidth=2)
        plt.plot([np.asarray(a).mean() for a in _accuracies], color='blue', linestyle='solid', marker='o', linewidth=2)
        plt.savefig('./train_{}_e{}_m{}_l{}.png'.format(loss_type, epochs, batch_shape[0], batch_shape[1]))
        
    def test(self, dataset, batch_shape=(64, 400)):
        print( 'Test...' )
        xtest = np.load( "{}nltk_xtest.npy".format(dataset) )
        ytest = np.load( "{}nltk_ytest.npy".format(dataset) )
        ytest = np.reshape(ytest, newshape=(ytest.shape[0], 1))
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='test_labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='test_batch')

        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        
        accuracies = list()
        bar = ProgressBar(max_value=np.floor(len(ytest)/batch_shape[0]).astype('i'))
        minibatch = enumerate(self.get_minibatch(xtest, ytest, ul=None, batch_shape=batch_shape))
        for i, test_batch in minibatch:
            fd = {batch: test_batch['x'], labels: test_batch['y'], K.backend.learning_phase(): 0} #test mode
            accuracies.append( self.sess.run(accuracy, feed_dict=fd) )
            
            bar.update(i)
        
        print( "\nAverage accuracy is {:.3f}".format(np.asarray(accuracies).mean()) )

In [ ]:
data = '../dataset/imdb/'
n_epochs = 10
n_ex = 64
ex_len = 400
lt = 'none'
pm = 0.02

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

embedding_weights = np.load( "{}nltk_embedding_matrix.npy".format(data) )

net = Network(session, embedding_weights)
net.train(data, batch_shape=(n_ex, ex_len), epochs=n_epochs, loss_type=lt, p_mult=pm, init=None, save=None)
net.test(data, batch_shape=(n_ex, ex_len))
    
K.backend.clear_session()

### Bit modification to the model

In [ ]:
class Network:
    def __init__(self, session, dict_weight, dropout=0.3, lstm_units=1024, dense_units=60):
        self.sess = session
        K.backend.set_session(self.sess)
        #defining layers
        dict_shape = dict_weight.shape
        self.emb = K.layers.Embedding(dict_shape[0], dict_shape[1], weights=[dict_weight], trainable=False, name='embedding')
        self.drop = K.layers.Dropout(rate=dropout, seed=91, name='dropout')
        self.lstm = K.layers.LSTM(lstm_units, stateful=False, return_sequences=False, name='lstm')
        
        self.dense = K.layers.Dense(dense_units, activation='relu', name='dense')
        self.p = K.layers.Dense(1, activation='sigmoid', name='p')
        #defining optimizer
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.0005)

    def __call__(self, batch, perturbation=None):  
        batch1 = batch[:,0:400]
        batch2 = batch[:,400:800]
        batch3 = batch[:,800:1200]
        
        embedding1 = self.emb(batch1)
        embedding2 = self.emb(batch2)
        embedding3 = self.emb(batch3)
        
        drop1 = self.drop(embedding1)
        drop2 = self.drop(embedding2)
        drop3 = self.drop(embedding3)
            
        if (perturbation is not None):
            drop1 += perturbation[0]
            drop2 += perturbation[1]
            drop3 += perturbation[2]
        
        lstm1 = self.lstm(drop1)
        lstm2 = self.lstm(drop2)
        lstm3 = self.lstm(drop3)
        lstm = tf.concat([lstm1, lstm2, lstm3], axis=1)
        dense = self.dense(lstm)
        
        return self.p(dense), (embedding1, embedding2, embedding3)
    
    def get_minibatch(self, x, y, ul, batch_shape=(16, 1200)):
        x = K.preprocessing.sequence.pad_sequences(x, maxlen=batch_shape[1])
        permutations = np.random.permutation( len(y) )
        ul_permutations = None
        len_ratio = None
        if (ul is not None):
            ul = K.preprocessing.sequence.pad_sequences(ul, maxlen=batch_shape[1])
            ul_permutations = np.random.permutation(len(ul))
            len_ratio = len(ul)/len(y)
        for s in range(0, len(y), batch_shape[0]):
            perm = permutations[s:s+batch_shape[0]]
            minibatch = {'x': x[perm], 'y': y[perm]}
            if (ul is not None):
                ul_perm = ul_permutations[int(np.floor(len_ratio*s)):int(np.floor(len_ratio*(s+batch_shape[0])))]
                minibatch.update( {'ul': np.concatenate((ul[ul_perm], x[perm]), axis=0)} )             
            yield minibatch
    
    def get_loss(self, batch, labels):
        pred, emb = self(batch)
        loss = K.losses.binary_crossentropy(labels, pred)
        return tf.reduce_mean( loss ), emb
    
    def get_adv_loss(self, batch, labels, loss, emb, p_mult):
        g1 = tf.gradients(loss, emb[0], aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
        g2 = tf.gradients(loss, emb[1], aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
        g3 = tf.gradients(loss, emb[2], aggregation_method=tf.AggregationMethod.EXPERIMENTAL_ACCUMULATE_N)[0]
        p_adv = list()
        p_adv.append( p_mult * tf.nn.l2_normalize(tf.stop_gradient(g1), dim=1) )
        p_adv.append( p_mult * tf.nn.l2_normalize(tf.stop_gradient(g2), dim=1) )
        p_adv.append( p_mult * tf.nn.l2_normalize(tf.stop_gradient(g3), dim=1) )
        adv_loss = K.losses.binary_crossentropy(labels, self(batch, p_adv)[0])
        return tf.reduce_mean( adv_loss )
    
    def validate(self, xval, yval, batch_shape=(64, 1200), log_path=None):
        print( 'Validation...' )
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='batch')

        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        accuracies = list()
        bar = ProgressBar(max_value=np.floor(len(yval)/batch_shape[0]).astype('i'))
        minibatch = enumerate(self.get_minibatch(xval, yval, None, batch_shape=batch_shape))
        for i, val_batch in minibatch:
            fd = {batch: val_batch['x'], labels: val_batch['y'], K.backend.learning_phase(): 0} #test mode
            accuracies.append( self.sess.run(accuracy, feed_dict=fd) )
            bar.update(i)
        
        log_msg = "\nValidation Average accuracy is {:.3f} -- batch shape {}"
        print( log_msg.format(np.asarray(accuracies).mean(), batch_shape) )
        log = None
        if(log_path is not None):
            log = open(log_path, 'a')
            log.write(log_msg.format(np.asarray(accuracies).mean(), batch_shape)+'\n')
            log.close()
    
    def train(self, dataset, batch_shape=(64, 1200), epochs=10, loss_type='none', p_mult=0.02, save=None, log_path=None):
        print( 'Training...' )
        xtrain = np.load( "{}nltk_xtrain.npy".format(dataset) )
        ytrain = np.load( "{}nltk_ytrain.npy".format(dataset) )
        
        xval = list()
        yval = list()
        for i in range(int(len(ytrain)*0.025)):
            xval.append( xtrain[0] ); xval.append( xtrain[-1] )
            yval.append( ytrain[0] ); yval.append( ytrain[-1] )
            xtrain = np.delete(xtrain, 0); xtrain = np.delete(xtrain, -1)
            ytrain = np.delete(ytrain, 0); ytrain = np.delete(ytrain, -1)
        xval = np.asarray(xval)
        yval = np.asarray(yval)
        
        yval = np.reshape(yval, newshape=(yval.shape[0],1))
        ytrain = np.reshape(ytrain, newshape=(ytrain.shape[0], 1))
        
        ultrain = np.load( "{}nltk_ultrain.npy".format(dataset) ) if (loss_type == 'v_adv') else None 
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='batch')
        ul_batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='ul_batch')
        
        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        loss, emb = self.get_loss(batch, labels)
        if (loss_type == 'adv'):
            loss += self.get_adv_loss(batch, labels, loss, emb, p_mult)
        elif (loss_type == 'v_adv'):
            loss += self.get_v_adv_loss(ul_batch, p_mult)

        opt = self.optimizer.minimize( loss )
        #initializing parameters
        self.sess.run( [var.initializer for var in tf.global_variables() if not('embedding' in var.name)] )
                       
        _losses = list()
        _accuracies = list()
        log = None
                         
        list_ratio = (len(ultrain)/len(ytrain)) if (ultrain is not None) else None
        for epoch in range(epochs):
            losses = list()
            accuracies = list()
            
            bar = ProgressBar(max_value=np.floor(len(ytrain)/batch_shape[0]).astype('i'))
            minibatch = enumerate(self.get_minibatch(xtrain, ytrain, ultrain, batch_shape=batch_shape))
            for i, train_batch in minibatch:
                fd = {batch: train_batch['x'], labels: train_batch['y'], K.backend.learning_phase(): 1} #training mode
                if (loss_type == 'v_adv'):
                    fd.update( {ul_batch: train_batch['ul']} )
                    
                _, acc_val, loss_val = self.sess.run([opt, accuracy, loss], feed_dict=fd)
                
                accuracies.append( acc_val )
                losses.append( loss_val )
                bar.update(i)
            
            _losses.append(losses)
            _accuracies.append(accuracies)
            
            log_msg = "\nEpoch {} of {} -- average accuracy is {:.3f} -- average loss is {:.3f}"
            print( log_msg.format(epoch+1, epochs, np.asarray(accuracies).mean(), np.asarray(losses).mean()) )
            if(log_path is not None):
                log = open(log_path, 'a')
                log.write(log_msg.format(epoch+1, epochs, np.asarray(accuracies).mean(), np.asarray(losses).mean())+'\n')
                log.close()
        
            #validation
            self.validate(xval, yval, batch_shape=batch_shape, log_path=log_path)
        
    def test(self, dataset, batch_shape=(64, 1200), log_path=None):
        print( 'Test...' )
        xtest = np.load( "{}nltk_xtest.npy".format(dataset) )
        ytest = np.load( "{}nltk_ytest.npy".format(dataset) )
        ytest = np.reshape(ytest, newshape=(ytest.shape[0], 1))
        
        labels = tf.placeholder(tf.float32, shape=(None, 1), name='labels')
        batch = tf.placeholder(tf.float32, shape=(None, batch_shape[1]), name='batch')

        accuracy = tf.reduce_mean( K.metrics.binary_accuracy(labels, self(batch)[0]) )
        
        accuracies = list()
        bar = ProgressBar(max_value=np.floor(len(ytest)/batch_shape[0]).astype('i'))
        minibatch = enumerate(self.get_minibatch(xtest, ytest, None, batch_shape=batch_shape))
        for i, test_batch in minibatch:
            fd = {batch: test_batch['x'], labels: test_batch['y'], K.backend.learning_phase(): 0} #test mode
            accuracies.append( self.sess.run(accuracy, feed_dict=fd) )
            bar.update(i)
        
        log_msg = "\nTest Average accuracy is {:.3f} -- batch shape {}"
        print( log_msg.format(np.asarray(accuracies).mean(), batch_shape) )
        
        log = None
        if(log_path is not None):
            log = open(log_path, 'a')
            log.write(log_msg.format(np.asarray(accuracies).mean(), batch_shape)+'\n')
            log.close()

In [ ]:
data = '../dataset/imdb/'
n_epochs = 10
n_ex = 16
ex_len = 1200
lt = 'none'
pm = 0.02

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

embedding_weights = np.load( "{}nltk_embedding_matrix.npy".format(data) )

# log text file
net_tp = 'baseline' if (lt == 'none') else lt
log_path = './pyramidal_{}_bs_{}_ep_{}_sl_{}.txt'.format(net_tp, n_ex, n_epochs, ex_len)
log = open(log_path, 'w')
log.write('Fancy Network with {} loss, {} epochs, {} batch size, {} maximum string length \n'.format(net_tp, n_epochs, n_ex, ex_len))
log.close()
                                  
net = Network(session, embedding_weights)
net.train(data, batch_shape=(n_ex, ex_len), epochs=n_epochs, loss_type=lt, p_mult=pm, log_path=log_path)
net.test(data, batch_shape=(n_ex, ex_len), log_path=log_path)
       
K.backend.clear_session()